In [1]:
!pip install vaderSentiment --quiet
!pip install textstat --quiet
!pip install nltk --quiet
!pip install tpot --quiet
!pip install seaborn --quiet

In [2]:
import pandas as pd
import numpy as np
import pickle
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem.porter import *
import string
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as VS
from textstat.textstat import *
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
import seaborn

from config import LSHS_DATAFILE, gpt_filtered_rephrase_lshs_file

%matplotlib inline

In [3]:
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /home/atif/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/atif/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
# import logging

# logging.basicConfig(
#     format='%(asctime)s %(levelname)-8s %(message)s',
#     level=logging.INFO,
#     datefmt='%Y-%m-%d %H:%M:%S')
# # The default levels are DEBUG, INFO, WARNING, ERROR, and CRITICAL.
# print(logging.WARNING)
# logging.getLogger()

In [5]:
import json
gpt_filtered_rephrase_tweets_file = gpt_filtered_rephrase_lshs_file
out_file = open(gpt_filtered_rephrase_tweets_file, "r")
filtered_rephrase_tweet_gpt = json.load(out_file)
out_file.close()

In [6]:
df = pd.read_csv(LSHS_DATAFILE)
domains = df['Domain'].unique().tolist()
for d in domains:
    print(d, df[df['Domain'] == d].shape)

Gender (9454, 4)
Religion (10869, 4)
Race (12013, 4)
Politics (11018, 4)
Sports (12306, 4)


In [7]:
df.head()

,Domain,TweetID,Tweet,Label
0,Gender,1344706773245038592,WATCH: Video previews #SurreyBC-shot film to f...,0
1,Gender,1344706877217792005,Men and women don’t have to solve their proble...,0
2,Gender,1344707261155962880,"At last I awake, very queer about the head, as...",0
3,Gender,1344707529213792256,WATCH: Video previews Surrey-shot film to focu...,0
4,Gender,1344709019865403394,heteronormativity is killing my people. how so...,1


In [8]:

stopwords=stopwords = nltk.corpus.stopwords.words("english")

other_exclusions = ["#ff", "ff", "rt"]
stopwords.extend(other_exclusions)

stemmer = PorterStemmer()


def preprocess(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    return parsed_text

def tokenize(tweet):
    """Removes punctuation & excess whitespace, sets to lowercase,
    and stems tweets. Returns a list of stemmed tokens."""


    #############LINE FIXED: * REPLACED WITH +##################### PREVIOUS::: tweet = " ".join(re.split("[^a-zA-Z.,!?]*", tweet.lower())).strip()
    tweet = " ".join(re.split("[^a-zA-Z]+", tweet.lower())).strip()
    tokens = [stemmer.stem(t) for t in tweet.split()]
    return tokens

def basic_tokenize(tweet):
    """Same as tokenize but without the stemming"""

    #############LINE FIXED: * REPLACED WITH +##################### PREVIOUS::: tweet = " ".join(re.split("[^a-zA-Z.,!?]*", tweet.lower())).strip()
    tweet = " ".join(re.split("[^a-zA-Z.,!?]+", tweet.lower())).strip()
    return tweet.split()

In [9]:
#Now get other features
sentiment_analyzer = VS()

def count_twitter_objs(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE
    4) hashtags with HASHTAGHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned.
    
    Returns counts of urls, mentions, and hashtags.
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    hashtag_regex = '#[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, 'URLHERE', parsed_text)
    parsed_text = re.sub(mention_regex, 'MENTIONHERE', parsed_text)
    parsed_text = re.sub(hashtag_regex, 'HASHTAGHERE', parsed_text)
    return(parsed_text.count('URLHERE'),parsed_text.count('MENTIONHERE'),parsed_text.count('HASHTAGHERE'))

def other_features(tweet):
    """This function takes a string and returns a list of features.
    These include Sentiment scores, Text and Readability scores,
    as well as Twitter specific features"""
    sentiment = sentiment_analyzer.polarity_scores(tweet)
    
    words = preprocess(tweet) #Get text only
    
    syllables = textstat.syllable_count(words)
    num_chars = sum(len(w) for w in words)
    num_chars_total = len(tweet)
    num_terms = len(tweet.split())
    num_words = len(words.split())
    avg_syl = round(float((syllables+0.001))/float(num_words+0.001),4)
    num_unique_terms = len(set(words.split()))
    
    ###Modified FK grade, where avg words per sentence is just num words/1
    FKRA = round(float(0.39 * float(num_words)/1.0) + float(11.8 * avg_syl) - 15.59,1)
    ##Modified FRE score, where sentence fixed to 1
    FRE = round(206.835 - 1.015*(float(num_words)/1.0) - (84.6*float(avg_syl)),2)
    
    twitter_objs = count_twitter_objs(tweet)
    retweet = 0
    if "rt" in words:
        retweet = 1
    features = [FKRA, FRE,syllables, avg_syl, num_chars, num_chars_total, num_terms, num_words,
                num_unique_terms, sentiment['neg'], sentiment['pos'], sentiment['neu'], sentiment['compound'],
                twitter_objs[2], twitter_objs[1],
                twitter_objs[0], retweet]
    #features = pandas.DataFrame(features)
    return features

def get_feature_array(tweets):
    feats=[]
    for t in tweets:
        feats.append(other_features(t))
    return np.array(feats)

In [10]:
class Features:
    
    def __init__(self):
        self.vectorizer = TfidfVectorizer(
        tokenizer=tokenize,
        preprocessor=preprocess,
        ngram_range=(1, 3),
        stop_words=stopwords,
        use_idf=True,
        smooth_idf=False,
        norm=None,
        decode_error='replace',
        max_features=10000,
        min_df=5,
        max_df=0.75
        )
        
        #We can use the TFIDF vectorizer to get a token matrix for the POS tags
        self.pos_vectorizer = TfidfVectorizer(
        tokenizer=None,
        lowercase=False,
        preprocessor=None,
        ngram_range=(1, 3),
        stop_words=None,
        use_idf=False,
        smooth_idf=False,
        norm=None,
        decode_error='replace',
        max_features=5000,
        min_df=5,
        max_df=0.75,
        )
        
        self.other_features_names = ["FKRA", "FRE","num_syllables", "avg_syl_per_word", "num_chars", "num_chars_total", \
                        "num_terms", "num_words", "num_unique_words", "vader neg","vader pos","vader neu", \
                        "vader compound", "num_hashtags", "num_mentions", "num_urls", "is_retweet"]
        
    def __tfidf__(self, tweets, isTrain=True):
        #Construct tfidf matrix and get relevant scores
        if isTrain:
            return self.vectorizer.fit_transform(tweets).toarray()
        else:
            return self.vectorizer.transform(tweets).toarray()
    
    def __get_pos_tags__(self, tweets):
        #Get POS tags for tweets and save as a string
        tweet_tags = []
        for t in tweets:
            tokens = basic_tokenize(preprocess(t))
            tags = nltk.pos_tag(tokens)
            tag_list = [x[1] for x in tags]
            tag_str = " ".join(tag_list)
            tweet_tags.append(tag_str)
        return tweet_tags
    
    def __pos_tags__(self, tweets, isTrain=True):
        tweet_tags = self.__get_pos_tags__(tweets)
        
        #Construct POS TF matrix and get vocab dict
        if isTrain:
            return self.pos_vectorizer.fit_transform(pd.Series(tweet_tags)).toarray()
        else:
            return self.pos_vectorizer.transform(pd.Series(tweet_tags)).toarray()
    
    def get_features(self, tweets, isTrain=True):
        tfidf = self.__tfidf__(tweets, isTrain=isTrain)
        pos = self.__pos_tags__(tweets, isTrain=isTrain)
        self.feats = get_feature_array(tweets)
        
        #Now join them all up
        # recover ids for mapping
        # ids = np.array(tweets.index.to_list())
        # ids = ids.reshape(ids.shape[0], 1)
        # M = np.concatenate([ids, tfidf,pos,feats],axis=1)
        M = np.concatenate([tfidf, pos, self.feats],axis=1)
        
        X = pd.DataFrame(M)
        
        return X
        
    

In [11]:
# X, y = df[['Domain', 'Tweet']], df['Label'].astype(int)

In [12]:
# HateLabel	Final hate label decision 0-Normal, 1-Offensive, 2-Hate

# Our Strategy

In [13]:
CF_LABEL = 0
import random
random.seed(42)

In [14]:
print(df['Label'].value_counts())
problematic_df = df[df['Label']>0]
problematic_df.shape

Label
0    44874
1     9669
2     1117
Name: count, dtype: int64


(10786, 4)

In [15]:
df.shape

(55660, 4)

In [16]:
problematic_df.tail()

,Domain,TweetID,Tweet,Label
55655,Sports,1277315350254751747,"Fuck off Gayle, professional footballer and yo...",1
55656,Sports,1277319456071581698,Omo I hate mancity abeg. What is this fluid fo...,2
55657,Sports,1277316487271854082,I hate playing Manchester United again,2
55658,Sports,1277319975305445381,I'll get trolled to fuck but I'd give anything...,1
55659,Sports,1277311388759863296,Chelsea though! I think I hate football. 🙄 #mufc,2


In [17]:
def get_tweets():
    gpt_counterfactual_tweets = {}
    tot = problematic_df.shape[0]
    # print(tot)
    for i in range(0, tot):
        idx = problematic_df.iloc[i].name
        if str(i) in filtered_rephrase_tweet_gpt:
            gpt_counterfactual_tweets[idx] = filtered_rephrase_tweet_gpt[str(i)]
    return gpt_counterfactual_tweets
    
gpt_counterfactual_tweets = get_tweets()

In [18]:
len(gpt_counterfactual_tweets)

8135

In [19]:
import config
from nltk.tokenize import TweetTokenizer
import mosestokenizer
import numpy as np

def get_offensive_words():
    _df = pd.read_csv(config.en_swear_words_datafile, index_col=0)
    
    s = np.logical_or(_df['Level of offensiveness']=='Strongest words', _df['Level of offensiveness']=='Strong words')
    # display(_df[s]['Word'].to_list())
    wd_list = _df['Word'].to_list()
    
    _df = pd.read_csv(config.en_profanity_datafile, index_col=None)
    s = _df['severity_description'] == 'Severe'
    # wd_list.extend(_df[s]['text'].to_list())
    wd_list.extend(_df['text'].to_list())
    wd_list = set(map(str.lower, wd_list))
    return wd_list

offensive_wd_list = get_offensive_words()

In [20]:
def find_phrases(tokens, phrases):
    tokens = list(map(str.lower, tokens))
    """
    Find phrases in a list of sequential tokens.
    
    Args:
        tokens (list): List of sequential tokens.
        phrases (list): List of phrases to search for.
        
    Returns:
        A list of tuples containing the start and end index of each found phrase.
    """
    found_phrases = []
    
    for i in range(len(tokens)):
        for phrase in phrases:
            if tokens[i:i+len(phrase)] == phrase:
                found_phrases.append((i, i+len(phrase)-1))
    
    return found_phrases

def offensive_lexicon_used(t):
    tk = TweetTokenizer()
    detk = mosestokenizer.MosesDetokenizer('en')
    tk = tk.tokenize(t)
    # print(tk)
    phrase_index = find_phrases(tk, list(map(str.split, offensive_wd_list)))
    return len(phrase_index)

In [21]:
def get_counterfactual_tweets(data, labels, cf_label, single_cf_per_tweet=False, cf_size_prop_to_data=1.0, only_tweets_with_offensive_lexicon=True):
    tweets = []
    cnt =0 
    for idx in data.index:
        if idx in gpt_counterfactual_tweets:
            if (not only_tweets_with_offensive_lexicon) or offensive_lexicon_used(X[idx]):
                cnt += 1
                if not single_cf_per_tweet:
                    tweets.extend(gpt_counterfactual_tweets[idx])
                else:
                    tweets.append(gpt_counterfactual_tweets[idx][0])
    print('> Total Tweets used to generate counterfactuals ' + str(cnt))
    print('> Total counterfactuals added ' + str(len(tweets)))
    k = round(cf_size_prop_to_data * len(tweets))
    
    tweets = random.sample(tweets, k=k)
    print('> Counterfactual size ' + str(k) + ' at rate ' + str(cf_size_prop_to_data))
    cf_target = k*[cf_label]
    return pd.concat([data, pd.Series(tweets)], axis=0), pd.concat([labels, pd.Series(cf_target)], axis=0)

In [22]:
from tpot import TPOTClassifier

# winner tpot-pipeline 40
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive

import warnings
warnings.filterwarnings("ignore")

In [23]:
only_tweets_with_offensive_lexicon = False

In [24]:
from sklearn.model_selection import StratifiedKFold
for d in domains:
    sel_df = df[df['Domain'] == d]
    print(d, sel_df.shape)
    X, y = sel_df['Tweet'], sel_df['Label'].astype(int)
    skf = StratifiedKFold(n_splits=2, random_state=None)
    for splt_idx, (train_index , test_index) in enumerate(skf.split(X, y)):
        print(splt_idx)
        X_train , X_test = X.iloc[train_index], X.iloc[test_index]
        y_train , y_test = y.iloc[train_index] , y.iloc[train_index]
        _,_ = get_counterfactual_tweets(
                X_train, y_train, cf_label=CF_LABEL, single_cf_per_tweet=True, cf_size_prop_to_data=.1, only_tweets_with_offensive_lexicon=only_tweets_with_offensive_lexicon)

Gender (9454, 4)
0
> Total Tweets used to generate counterfactuals 845
> Total counterfactuals added 845
> Counterfactual size 84 at rate 0.1
1
> Total Tweets used to generate counterfactuals 767
> Total counterfactuals added 767
> Counterfactual size 77 at rate 0.1
Religion (10869, 4)
0
> Total Tweets used to generate counterfactuals 762
> Total counterfactuals added 762
> Counterfactual size 76 at rate 0.1
1
> Total Tweets used to generate counterfactuals 771
> Total counterfactuals added 771
> Counterfactual size 77 at rate 0.1
Race (12013, 4)
0
> Total Tweets used to generate counterfactuals 570
> Total counterfactuals added 570
> Counterfactual size 57 at rate 0.1
1
> Total Tweets used to generate counterfactuals 520
> Total counterfactuals added 520
> Counterfactual size 52 at rate 0.1
Politics (11018, 4)
0
> Total Tweets used to generate counterfactuals 1001
> Total counterfactuals added 1001
> Counterfactual size 100 at rate 0.1
1
> Total Tweets used to generate counterfactuals

In [25]:
from sklearn.metrics import accuracy_score, f1_score


def __exp__(pipeline, _x_train, _x_test, _y_train, _y_test, CF=False):
    f = Features()
    training_features = f.get_features(_x_train, isTrain=True)
    testing_features = f.get_features(_x_test, isTrain=False)
    
    if not CF:
        print('> Train samples', _x_train.shape[0])
    else:
        print('> Train with CF samples', _x_train.shape[0])
    
    try:
        # Fix random state for all the steps in exported pipeline
        set_param_recursive(pipeline.steps, 'random_state', 42)
    except:
        pass

    pipeline.fit(training_features, _y_train)
    results = pipeline.predict(testing_features)

    # report = classification_report(_y_test, results)
    # print(report)
    acc = accuracy_score(_y_test, results)
    f1_marco = f1_score(_y_test, results, average='macro')
    f1_weighted = f1_score(_y_test, results, average='weighted')
    f1_non_avg = f1_score(_y_test, results, average=None)
    r = {'Accuracy': acc,
         'F1-Macro': f1_marco,
         'F1-Weighted': f1_weighted,
         'F1_Class 0': f1_non_avg[0],
         'F1_Class 1': f1_non_avg[1],
         'F1_Class 2': f1_non_avg[2],
        }
    print(r)
    return [r]

def run_experiment_org(pipeline, n_splits=2):
    out_lst = []
    skf = StratifiedKFold(n_splits=n_splits, random_state=None)
    for splt_idx, (train_index , test_index) in enumerate(skf.split(X, y)):
        # print(splt_idx)
        x_train , x_test = X.iloc[train_index], X.iloc[test_index]
        training_target , testing_target = y.iloc[train_index] , y.iloc[test_index]

        org = __exp__(pipeline, x_train, x_test, training_target, testing_target, CF=False)

        l = [('splt_idx', splt_idx, len(x_test)),  ('train', len(x_train)), {'Org': org}]
        out_lst.append(l)
    return out_lst

def run_experiment_counter_factuals(pipeline, n_splits=2, cf_size_prop_to_data=0.1):
    out_lst = []
    skf = StratifiedKFold(n_splits=n_splits, random_state=None)
    for splt_idx, (train_index , test_index) in enumerate(skf.split(X, y)):
        # print(splt_idx)
        x_train , x_test = X.iloc[train_index], X.iloc[test_index]
        training_target , testing_target = y.iloc[train_index] , y.iloc[test_index]

        # org = __exp__(pipeline, x_train, x_test, training_target, testing_target, CF=False)

        x_train_with_cf, training_with_cf_target = get_counterfactual_tweets(
            x_train, training_target, cf_label=CF_LABEL, single_cf_per_tweet=True, cf_size_prop_to_data=cf_size_prop_to_data, only_tweets_with_offensive_lexicon=only_tweets_with_offensive_lexicon)

        cf = __exp__(pipeline, x_train_with_cf, x_test, training_with_cf_target, testing_target , CF=True)
        l = [('splt_idx', splt_idx, len(x_test)),  ('train', len(x_train_with_cf)), {'CF': cf}]
        out_lst.append(l)
    return out_lst

In [26]:
for d in domains:
    sel_df = df[df['Domain'] == d]
    print(d, sel_df.shape)
    X, y = sel_df['Tweet'], sel_df['Label'].astype(int)
    skf = StratifiedKFold(n_splits=3, random_state=None)
    for splt_idx, (train_index , test_index) in enumerate(skf.split(X, y)):
        print(splt_idx)
        X_train , X_test = X.iloc[train_index], X.iloc[test_index]
        training_target , testing_target = y.iloc[train_index] , y.iloc[test_index]
        print(X_train.shape, training_target.shape, X_test.shape, testing_target.shape)
        break
        _,_ = get_counterfactual_tweets(
            x_train, training_target, cf_label=CF_LABEL, single_cf_per_tweet=True, cf_size_prop_to_data=cf_size_prop_to_data, only_tweets_with_offensive_lexicon=only_tweets_with_offensive_lexicon)

Gender (9454, 4)
0
(6302,) (6302,) (3152,) (3152,)
Religion (10869, 4)
0
(7246,) (7246,) (3623,) (3623,)
Race (12013, 4)
0
(8008,) (8008,) (4005,) (4005,)
Politics (11018, 4)
0
(7345,) (7345,) (3673,) (3673,)
Sports (12306, 4)
0
(8204,) (8204,) (4102,) (4102,)


In [27]:
n_splits=5
cf_size_prop_to_data_lst = list(np.arange(0.1, 1.1, 0.1))
# n_splits=2
# cf_size_prop_to_data_lst = list(np.arange(0.1, 0.3, 0.1))


In [28]:
def get_tpot_pipeline():
    return make_pipeline(
        StackingEstimator(estimator=LinearSVC(C=15.0, dual=False, loss="squared_hinge", penalty="l2", tol=0.01)),
        DecisionTreeClassifier(criterion="gini", max_depth=3, min_samples_leaf=12, min_samples_split=3)
    )
out_dict = {}
for d in domains:
    out_dict[d] = {}
    sel_df = df[df['Domain'] == d]
    print(d, sel_df.shape)
    X, y = sel_df['Tweet'], sel_df['Label'].astype(int)
    
    tpot_exported_pipeline = get_tpot_pipeline()
    res_tpot_lst = run_experiment_org(tpot_exported_pipeline, n_splits=n_splits)
    out_dict[d]['Org'] = res_tpot_lst
    out_dict[d]['CF'] = {}
    for cf_size_prop_to_data in cf_size_prop_to_data_lst:
        tpot_exported_pipeline = get_tpot_pipeline()
        
        res_tpot_lst = run_experiment_counter_factuals(tpot_exported_pipeline, n_splits=n_splits, cf_size_prop_to_data=cf_size_prop_to_data)
        out_dict[d]['CF'][cf_size_prop_to_data] = res_tpot_lst
    json.dump(out_dict, open('out/tpot-lshs22.json', 'w'))

Gender (9454, 4)
> Train samples 7563
{'Accuracy': 0.8677948175568482, 'F1-Macro': 0.5809316618755357, 'F1-Weighted': 0.8507515378499279, 'F1_Class 0': 0.9249190938511327, 'F1_Class 1': 0.6439628482972135, 'F1_Class 2': 0.1739130434782609}
> Train samples 7563
{'Accuracy': 0.8958223162347964, 'F1-Macro': 0.6021672042777194, 'F1-Weighted': 0.8818566852903429, 'F1_Class 0': 0.937724160417346, 'F1_Class 1': 0.7437774524158125, 'F1_Class 2': 0.125}
> Train samples 7563
{'Accuracy': 0.9439450026441036, 'F1-Macro': 0.6939388704506445, 'F1-Weighted': 0.938223027421659, 'F1_Class 0': 0.9661933739012847, 'F1_Class 1': 0.8934010152284265, 'F1_Class 2': 0.2222222222222222}
> Train samples 7563
{'Accuracy': 0.9481755684822845, 'F1-Macro': 0.6656443948269043, 'F1-Weighted': 0.9408933346703915, 'F1_Class 0': 0.9682432432432433, 'F1_Class 1': 0.9074778200253485, 'F1_Class 2': 0.12121212121212122}
> Train samples 7564
{'Accuracy': 0.8174603174603174, 'F1-Macro': 0.5270614631141506, 'F1-Weighted': 0.82

In [29]:
from sklearn.pipeline import Pipeline

def get_davidson_pipeline():
    # CHANGE --> solver='liblinear' added for l1, otherwise won't work
    # CHANGE --> max_iter=10000 added for l2, otherwsie ConvergenceWarning: lbfgs failed to converge (status=1): STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
    return Pipeline([('sel-lg-l1', SelectFromModel(LogisticRegression(class_weight='balanced',penalty="l1",C=0.01, solver='liblinear'))),
                     ('lg-l2',LogisticRegression(class_weight='balanced',penalty='l2',C=0.01,  max_iter=10000))])
    
for d in domains:
    out_dict[d] = {}
    sel_df = df[df['Domain'] == d]
    print(d, sel_df.shape)
    X, y = sel_df['Tweet'], sel_df['Label'].astype(int)

    davidson_pipeline = get_davidson_pipeline()
    res_dav_lst = run_experiment_org(davidson_pipeline, n_splits=n_splits)
    out_dict[d]['Org'] = res_dav_lst
    out_dict[d]['CF'] = {}
    for cf_size_prop_to_data in cf_size_prop_to_data_lst:
        davidson_pipeline = get_davidson_pipeline()
    
        res_dav_lst = run_experiment_counter_factuals(davidson_pipeline, n_splits=n_splits, cf_size_prop_to_data=cf_size_prop_to_data)
        out_dict[d]['CF'][cf_size_prop_to_data] = res_dav_lst
    json.dump(out_dict, open('out/davidson-pipline-lshs22.json', 'w'))

Gender (9454, 4)
> Train samples 7563
{'Accuracy': 0.812797461660497, 'F1-Macro': 0.5694941927587891, 'F1-Weighted': 0.8382970005950483, 'F1_Class 0': 0.905241235682055, 'F1_Class 1': 0.6574923547400612, 'F1_Class 2': 0.14574898785425103}
> Train samples 7563
{'Accuracy': 0.8873611845584347, 'F1-Macro': 0.6798064356328309, 'F1-Weighted': 0.897388736652952, 'F1_Class 0': 0.9345279117849757, 'F1_Class 1': 0.812920592193809, 'F1_Class 2': 0.291970802919708}
> Train samples 7563
{'Accuracy': 0.9185616076150185, 'F1-Macro': 0.6731999128173992, 'F1-Weighted': 0.9314022991537707, 'F1_Class 0': 0.9515669515669515, 'F1_Class 1': 0.9180327868852459, 'F1_Class 2': 0.15}
> Train samples 7563
{'Accuracy': 0.952934955050238, 'F1-Macro': 0.7644835030001648, 'F1-Weighted': 0.9565906181184256, 'F1_Class 0': 0.9780717020536026, 'F1_Class 1': 0.9245742092457421, 'F1_Class 2': 0.3908045977011494}
> Train samples 7564
{'Accuracy': 0.7634920634920634, 'F1-Macro': 0.5566700304976218, 'F1-Weighted': 0.7864251

In [30]:
print('done', cf_size_prop_to_data)

done 1.0


In [32]:
0.1 > 0.05

True